The purpose of this notebook is to identify the frequency of conflicts between metrics that purport to measure the degree to which a plan favors one party or another.

In [2]:
from typing import List, Dict, Any, Set

import os
import pandas as pd
from collections import defaultdict

from rdametrics import states, chambers, ensembles

Load the scores dataframe

In [3]:
scores_path: str = "~/local/beta-ensembles/prepackaged/scores/scores.parquet"
scores_df = pd.read_parquet(os.path.expanduser(scores_path))

Helper code

In [4]:
ensembles = [e for e in ensembles if e not in ["A1", "A2", "A3", "A4", "Rev*"]]

partisan_advantage_metrics: List[str] = ["disproportionality", "efficiency_gap"]

symmetry_metrics: List[str] = [
    "geometric_seats_bias",
    "seats_bias",
    "votes_bias",
]

packing_cracking_metrics: List[str] = [
    "mean_median_average_district",
    "lopsided_outcomes",
    "declination",
]

partisan_metrics: List[str] = partisan_advantage_metrics + symmetry_metrics + packing_cracking_metrics

def same_sign(a, b):
    if a == 0.0 or b == 0.0:
        return True
    
    if pd.isna(a) or pd.isna(b):
        return True

    return a * b > 0

def same_signs(list1, list2):
    # Combine both lists and filter out zero values and NaN values
    all_values = list1 + list2
    non_zero_defined_values = [x for x in all_values if x != 0 and not pd.isna(x)]
    
    # If no valid values or only one valid value, return True
    if len(non_zero_defined_values) <= 1:
        return True
    
    # Check if all valid values have the same sign
    all_positive = all(x > 0 for x in non_zero_defined_values)
    all_negative = all(x < 0 for x in non_zero_defined_values)
    
    return all_positive or all_negative



Find examples of IL EG/PR differences

In [6]:
IL_eg_vs_pr: int = 0
combos: Set = set()
examples: List = list()

for index, row in scores_df.iterrows():
    if row["ensemble"] not in ensembles:
        continue

    xx, chamber, ensemble = (row["state"], row["chamber"], row["ensemble"])
    combo = (xx, chamber, ensemble)
                
    if xx == "IL" and not same_sign(row["disproportionality"], row["efficiency_gap"]) and combo not in combos:
        IL_eg_vs_pr += 1
        if len(examples) < 33:
            combos.add(combo) 
            examples.append(f"{combo} / {row['map']:09d}")
        else:
            break

print("Examples of IL EG/PR differences:")
for example in examples:
    print(f" - {example}")

Examples of IL EG/PR differences:
 - ('IL', 'congress', 'A0') / 000002500
 - ('IL', 'congress', 'Pop-') / 000005000
 - ('IL', 'congress', 'Pop+') / 000002500
 - ('IL', 'congress', 'B') / 000002500
 - ('IL', 'congress', 'C') / 000002500
 - ('IL', 'congress', 'D') / 000002500
 - ('IL', 'congress', 'R25') / 000007500
 - ('IL', 'congress', 'R50') / 000002500
 - ('IL', 'congress', 'R75') / 000002500
 - ('IL', 'congress', 'R100') / 000002500
 - ('IL', 'upper', 'A0') / 000002500
 - ('IL', 'upper', 'Pop-') / 000010000
 - ('IL', 'upper', 'Pop+') / 000002500
 - ('IL', 'upper', 'B') / 000005000
 - ('IL', 'upper', 'C') / 000002500
 - ('IL', 'upper', 'D') / 000005000
 - ('IL', 'upper', 'R25') / 000002500
 - ('IL', 'upper', 'R50') / 000002500
 - ('IL', 'upper', 'R75') / 000002500
 - ('IL', 'upper', 'R100') / 000012500
 - ('IL', 'lower', 'A0') / 000002500
 - ('IL', 'lower', 'Pop-') / 000002500
 - ('IL', 'lower', 'Pop+') / 000002500
 - ('IL', 'lower', 'B') / 000010000
 - ('IL', 'lower', 'C') / 0000050

Find examples where seat and vote bias are different wrto PR

In [7]:
combos: Set = set()
examples: List = list()

for index, row in scores_df.iterrows():
    if row["ensemble"] not in ensembles:
        continue

    xx, chamber, ensemble = (row["state"], row["chamber"], row["ensemble"])
    combo = (xx, chamber, ensemble)

    sb: bool = False
    vb: bool = False

    if not same_sign(row["seats_bias"], row["disproportionality"]):
        sb = True
    if not same_sign(row["votes_bias"], row["disproportionality"]):
        vb = True

    if (sb and not vb) or (vb and not sb):
        if len(examples) < 10:
            combos.add(combo) 
            examples.append(f"{combo} / {row['map']:09d} -- a_s: {row['seats_bias']}, a_v: {row['votes_bias']}, PR: {row['disproportionality']}")
        else:
            break

print("Examples of seats/votes bias differences:")
for example in examples:
    print(f" - {example}")

Examples of seats/votes bias differences:
 - ('FL', 'congress', 'A0') / 001000000 -- a_s: -0.0001, a_v: -0.0, PR: 0.0274
 - ('FL', 'congress', 'A0') / 013507500 -- a_s: -0.0001, a_v: -0.0, PR: 0.0255
 - ('FL', 'congress', 'A0') / 013890000 -- a_s: -0.0001, a_v: -0.0, PR: 0.0312
 - ('FL', 'congress', 'A0') / 017857500 -- a_s: -0.0001, a_v: -0.0, PR: 0.0251
 - ('FL', 'congress', 'A0') / 021332500 -- a_s: -0.0001, a_v: -0.0, PR: 0.022
 - ('FL', 'congress', 'A0') / 026645000 -- a_s: -0.0001, a_v: -0.0, PR: 0.0277
 - ('FL', 'congress', 'A0') / 031257500 -- a_s: -0.0001, a_v: -0.0, PR: 0.0371
 - ('FL', 'congress', 'A0') / 034045000 -- a_s: -0.0001, a_v: -0.0, PR: 0.0311
 - ('FL', 'congress', 'A0') / 041297500 -- a_s: -0.0001, a_v: -0.0, PR: 0.0302
 - ('FL', 'congress', 'A0') / 046722500 -- a_s: -0.0001, a_v: -0.0, PR: 0.0353


Count of rows with undefined declination. Get examples.

In [6]:
total: int = 0
undefined_decl: int = 0
combos: Set = set()
examples: List = list()

for index, row in scores_df.iterrows():
    if row["ensemble"] not in ensembles:
        continue

    total += 1
    if pd.isna(row["declination"]):
        undefined_decl += 1

        if len(examples) < 10:
            combo = (row["state"], row["chamber"], row["ensemble"])
            # if row["state"] not in states:
            if combo not in combos:
                combos.add(combo) 
                examples.append(f"{combo} / {row['map']:09d}")

# print(f"# of plans w/ undefined declination: {undefined_decl:,} of {total:,} ({undefined_decl/total:.2%})")
print(f"# of *all* plans w/ undefined declination: {undefined_decl:,} of {total:,} ({undefined_decl/total:.2%})")
print(f"Affected combos ({len(combos)}): {combos}")
affected: int = len(combos) * 20000
print(f"# of *affected* plans w/ undefined declination: {undefined_decl:,} of {affected:,} ({undefined_decl/affected:.2%})")
print("Examples of undefined declination plans:")
for example in examples:
    print(f" - {example}")


# of *all* plans w/ undefined declination: 75,736 of 4,619,979 (1.64%)
Affected combos (10): {('NY', 'congress', 'R75'), ('NY', 'congress', 'R100'), ('NY', 'congress', 'C'), ('NY', 'congress', 'D'), ('NY', 'congress', 'Pop-'), ('NY', 'congress', 'A0'), ('NY', 'congress', 'Pop+'), ('NY', 'congress', 'B'), ('NY', 'congress', 'R50'), ('NY', 'congress', 'R25')}
# of *affected* plans w/ undefined declination: 75,736 of 200,000 (37.87%)
Examples of undefined declination plans:
 - ('NY', 'congress', 'A0') / 000025000
 - ('NY', 'congress', 'Pop-') / 000005000
 - ('NY', 'congress', 'Pop+') / 000002500
 - ('NY', 'congress', 'B') / 000007500
 - ('NY', 'congress', 'C') / 000005000
 - ('NY', 'congress', 'D') / 000005000
 - ('NY', 'congress', 'R25') / 000005000
 - ('NY', 'congress', 'R50') / 000002500
 - ('NY', 'congress', 'R75') / 000002500
 - ('NY', 'congress', 'R100') / 000002500


Calculate average declination for NY/congress as a point of comparison

In [11]:
import numpy as np
from data.helpers import arr_from_scores

all_decls = list()
for e in ensembles:
    arr = arr_from_scores("NY", "congress", e, "declination", scores_df)
    all_decls.extend(arr)

all_decls = [d for d in all_decls if not np.isnan(d)]
mean_decl = np.mean(all_decls)

print (f"{(mean_decl, all_decls.count(np.nan), len(all_decls))}")
all_decls

(18.434526101633825, 0, 144263)


[24.0578,
 22.3725,
 24.2011,
 22.2334,
 20.8746,
 16.2627,
 15.9999,
 12.3575,
 18.4133,
 24.3983,
 18.8728,
 16.9127,
 23.3133,
 14.0739,
 19.9922,
 11.6493,
 14.7116,
 14.0237,
 17.4566,
 19.5858,
 12.0968,
 14.8658,
 20.3529,
 24.6782,
 18.7627,
 22.3833,
 17.7541,
 16.2376,
 22.1042,
 22.4949,
 7.5743,
 15.6806,
 21.5919,
 20.5419,
 23.8428,
 17.7482,
 24.0183,
 21.6678,
 7.4812,
 16.2376,
 11.6057,
 17.2807,
 19.8951,
 23.5577,
 19.5428,
 24.0784,
 23.4927,
 20.3803,
 18.2057,
 12.3788,
 21.0905,
 22.161,
 15.7705,
 14.1685,
 17.243,
 17.4549,
 10.6934,
 16.0126,
 12.7519,
 23.5532,
 23.0068,
 21.9365,
 23.3116,
 16.4538,
 20.2513,
 13.5867,
 13.9695,
 12.0899,
 16.6817,
 23.3093,
 23.3279,
 9.197,
 17.8717,
 23.6509,
 20.1221,
 14.4506,
 18.9883,
 24.4536,
 18.4164,
 18.5496,
 22.2741,
 17.6333,
 21.1439,
 20.5907,
 23.0866,
 18.2116,
 24.6256,
 22.4323,
 17.0729,
 20.7183,
 22.9844,
 15.6202,
 15.9884,
 22.0589,
 23.8865,
 4.6824,
 17.7331,
 22.4665,
 13.5362,
 11.9589,
 20.618

Setup counters for various kinds of conflicts.

In [4]:
by_state: Dict[str, Any] = dict()

for xx in states:
    by_state[xx] = {
        "Vf": None,
        "total": 0
    }

conflicts: Dict[str, Any] = dict()
categories: List[str] = ["with_proportionality", "within_symmetry", "within_packing_cracking", "between_symmetry_and_packing_cracking"]
subcategories: Dict[str, List[str]] = {
    "within_symmetry": ["all", "seats_vs_votes"],
    "within_packing_cracking": ["all"],
    "between_symmetry_and_packing_cracking": ["all"]
}

conflicts["with_proportionality"] = dict()
for sc in partisan_metrics[1:]:
    conflicts["with_proportionality"][sc] = dict()
    for xx in states:
        for chamber in chambers:
            for ensemble in ensembles:
                combo = (xx, chamber, ensemble)
                conflicts["with_proportionality"][sc][combo] = {
                    "count": 0,
                    "example": None,
                    #
                    "value": None,
                    "disproportionality": None,
                    "delta": None,
                }

for c in categories[1:]:
    conflicts[c] = dict()
    for sc in subcategories[c]:
        conflicts[c][sc] = dict()
        for xx in states:
            for chamber in chambers:
                for ensemble in ensembles:
                    combo = (xx, chamber, ensemble)
                    conflicts[c][sc][combo] = {
                        "count": 0,
                        "example": None,
                    }

Count differences for seat bias, vote bias, and declination -- 3 of the "Big 5" metrics

In [12]:
big_5_diffs: Dict[str, Any] = dict()
FL_diffs: int = 0

for xx in states:
    big_5_diffs[xx] = {
        "Vf": None,
        "total": 0,
        "count": 0,
        "conflict-rate": 0.0
    }

for index, row in scores_df.iterrows():
    if row["ensemble"] not in ensembles:
        continue

    xx, chamber, ensemble = (row["state"], row["chamber"], row["ensemble"])
    combo = (xx, chamber, ensemble)

    big_5_diffs[xx]["total"] += 1
    if big_5_diffs[xx]["Vf"] is None:
        big_5_diffs[xx]["Vf"] = row["estimated_vote_pct"]

    if not same_sign(row["seats_bias"], row["votes_bias"]) or not same_sign(row["votes_bias"], row["declination"]) or not same_sign(row["seats_bias"], row["declination"]):
        big_5_diffs[xx]["count"] += 1
        if xx == "FL":
            FL_diffs += 1
            if FL_diffs <= 10:
                print(f"FL diffs: {combo} / {row['map']:09d}")
        continue

for xx in states:
    big_5_diffs[xx]["conflict-rate"] = big_5_diffs[xx]["count"] / big_5_diffs[xx]["total"]

big_5_diffs

FL diffs: ('FL', 'congress', 'A0') / 000035000
FL diffs: ('FL', 'congress', 'A0') / 000090000
FL diffs: ('FL', 'congress', 'A0') / 000122500
FL diffs: ('FL', 'congress', 'A0') / 000160000
FL diffs: ('FL', 'congress', 'A0') / 000230000
FL diffs: ('FL', 'congress', 'A0') / 000242500
FL diffs: ('FL', 'congress', 'A0') / 000285000
FL diffs: ('FL', 'congress', 'A0') / 000297500
FL diffs: ('FL', 'congress', 'A0') / 000357500
FL diffs: ('FL', 'congress', 'A0') / 000405000


{'FL': {'Vf': 0.4837,
  'total': 659997,
  'count': 92542,
  'conflict-rate': 0.14021578885964633},
 'IL': {'Vf': 0.5817,
  'total': 659997,
  'count': 32434,
  'conflict-rate': 0.04914264761809523},
 'MI': {'Vf': 0.5188,
  'total': 659997,
  'count': 135,
  'conflict-rate': 0.00020454638430174681},
 'NC': {'Vf': 0.4943,
  'total': 659997,
  'count': 4274,
  'conflict-rate': 0.006475787011153081},
 'NY': {'Vf': 0.6478,
  'total': 659997,
  'count': 1016,
  'conflict-rate': 0.0015394009366709242},
 'OH': {'Vf': 0.4638,
  'total': 659997,
  'count': 31338,
  'conflict-rate': 0.0474820340092455},
 'WI': {'Vf': 0.5068,
  'total': 659997,
  'count': 1592,
  'conflict-rate': 0.0024121321763583775}}

Count instances where the scores conflict. Keep track by state / chamber / ensemble combination.

In [5]:
OH_packing_cracking_conflicts: int = 0
for index, row in scores_df.iterrows():
    if row["ensemble"] not in ensembles:
        continue

    xx, chamber, ensemble = (row["state"], row["chamber"], row["ensemble"])
    combo = (xx, chamber, ensemble)

    by_state[xx]["total"] += 1
    if by_state[xx]["Vf"] is None:
        by_state[xx]["Vf"] = row["estimated_vote_pct"]

    packing_cracking_consistent: bool = True
    partisan_symmetry_consistent: bool = True

    if not same_sign(row["seats_bias"], row["geometric_seats_bias"]) or not same_sign(row["votes_bias"], row["geometric_seats_bias"]):
        conflicts["within_symmetry"]["all"][combo]["count"] += 1
        partisan_symmetry_consistent = False
        if conflicts["within_symmetry"]["all"][combo]["example"] is None:
            conflicts["within_symmetry"]["all"][combo]["example"] = f"{combo} / {row['map']:09d}"
    if not same_sign(row["seats_bias"], row["votes_bias"]):
        conflicts["within_symmetry"]["seats_vs_votes"][combo]["count"] += 1
        if conflicts["within_symmetry"]["seats_vs_votes"][combo]["example"] is None:
            conflicts["within_symmetry"]["seats_vs_votes"][combo]["example"] = f"{combo} / {row['map']:09d}"

    if not same_sign(row["mean_median_average_district"], row["lopsided_outcomes"]) or not same_sign(row["mean_median_average_district"], row["declination"]) or not same_sign(row["lopsided_outcomes"], row["declination"]):  
        conflicts["within_packing_cracking"]["all"][combo]["count"] += 1
        packing_cracking_consistent = False
        if conflicts["within_packing_cracking"]["all"][combo]["example"] is None:
            conflicts["within_packing_cracking"]["all"][combo]["example"] = f"{combo} / {row['map']:09d}"
        if xx == "OH":
            OH_packing_cracking_conflicts += 1
            if OH_packing_cracking_conflicts <= 10:
                print(f"OH packing/cracking conflict: {combo} / {row['map']:09d}")

    if packing_cracking_consistent and partisan_symmetry_consistent and not same_signs([row["seats_bias"], row["votes_bias"], row["geometric_seats_bias"]], [row["mean_median_average_district"], row["lopsided_outcomes"], row["declination"]]):
        conflicts["between_symmetry_and_packing_cracking"]["all"][combo]["count"] += 1
        if conflicts["between_symmetry_and_packing_cracking"]["all"][combo]["example"] is None:
            conflicts["between_symmetry_and_packing_cracking"]["all"][combo]["example"] = f"{combo} / {row['map']:09d}"

    for i, sc in enumerate(partisan_metrics[1:]):
        if same_sign(row[sc], row["disproportionality"]):
            continue
        
        else:
            conflicts["with_proportionality"][sc][combo]["count"] += 1
            delta: float = abs(row[sc] - row["disproportionality"])

            if conflicts["with_proportionality"][sc][combo]["example"] is None or delta > conflicts["with_proportionality"][sc][combo]["delta"]:
                conflicts["with_proportionality"][sc][combo]["example"] = f"{combo} / {row['map']:09d}"
                conflicts["with_proportionality"][sc][combo]["value"] = row[sc]
                conflicts["with_proportionality"][sc][combo]["disproportionality"] = row["disproportionality"]
                conflicts["with_proportionality"][sc][combo]["delta"] = delta

OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000002500
OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000005000
OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000012500
OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000017500
OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000022500
OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000025000
OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000027500
OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000035000
OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000042500
OH packing/cracking conflict: ('OH', 'congress', 'A0') / 000045000


Aggregate the results by state

In [6]:
conflicts_summary: Dict[str, Any] = dict()

for c in categories[1:]:
    conflicts_summary[c] = dict()
    for sc in subcategories[c]:
        conflicts_summary[c][sc] = dict()
        for xx in states:
            conflicts_summary[c][sc][xx] = {
                "count": 0,
                "conflict-rate": None,
                "example": None,
            }

conflicts_summary["with_proportionality"] = dict()
for sc in partisan_metrics[1:]:
    conflicts_summary["with_proportionality"][sc] = dict()
    for xx in states:
        conflicts_summary["with_proportionality"][sc][xx] = {
            "count": 0,
            "conflict-rate": None,
            "example": None,
            "value": None,
            "disproportionality": None,
            "delta": None,
        }

for sc in partisan_metrics[1:]:
    for combo, _data in conflicts["with_proportionality"][sc].items():
        xx, chamber, ensemble = combo
        conflicts_summary["with_proportionality"][sc][xx]["count"] += _data["count"]
        if conflicts_summary["with_proportionality"][sc][xx]["example"] is None or (_data["delta"] is not None and (_data["delta"] > conflicts_summary["with_proportionality"][sc][xx]["delta"])):
            conflicts_summary["with_proportionality"][sc][xx]["example"] = _data["example"]
            conflicts_summary["with_proportionality"][sc][xx]["value"] = _data["value"]
            conflicts_summary["with_proportionality"][sc][xx]["disproportionality"] = _data["disproportionality"]
            conflicts_summary["with_proportionality"][sc][xx]["delta"] = _data["delta"]

for sc in partisan_metrics[1:]:
    for xx in states:
        conflicts_summary["with_proportionality"][sc][xx]["conflict-rate"] = conflicts_summary["with_proportionality"][sc][xx]["count"] / by_state[xx]["total"]

for c in categories[1:]:
    for sc in subcategories[c]:
        for combo, _data in conflicts[c][sc].items():
            xx, chamber, ensemble = combo
            conflicts_summary[c][sc][xx]["count"] += _data["count"]
            if conflicts_summary[c][sc][xx]["example"] is None:
                conflicts_summary[c][sc][xx]["example"] = _data["example"]

for c in categories[1:]:
    for sc in subcategories[c]:
        for xx in states:
            conflicts_summary[c][sc][xx]["conflict-rate"] = conflicts_summary[c][sc][xx]["count"] / by_state[xx]["total"]


In [15]:
conflicts_summary

{'within_symmetry': {'all': {'FL': {'count': 23783,
    'conflict-rate': 0.03603501228035885,
    'example': "('FL', 'congress', 'A0') / 000022500"},
   'IL': {'count': 101,
    'conflict-rate': 0.00015303099862575134,
    'example': "('IL', 'congress', 'A0') / 009895000"},
   'MI': {'count': 64,
    'conflict-rate': 9.697013774305035e-05,
    'example': "('MI', 'congress', 'A0') / 004472500"},
   'NC': {'count': 191,
    'conflict-rate': 0.0002893952548269159,
    'example': "('NC', 'congress', 'A0') / 002597500"},
   'NY': {'count': 948,
    'conflict-rate': 0.0014363701653189332,
    'example': "('NY', 'congress', 'A0') / 014170000"},
   'OH': {'count': 18048,
    'conflict-rate': 0.0273455788435402,
    'example': "('OH', 'congress', 'A0') / 000005000"},
   'WI': {'count': 48,
    'conflict-rate': 7.272760330728776e-05,
    'example': "('WI', 'congress', 'A0') / 003940000"}},
  'seats_vs_votes': {'FL': {'count': 0, 'conflict-rate': 0.0, 'example': None},
   'IL': {'count': 0, 'conf

Format for reporting & plotting

In [17]:
report: Dict[str, Dict[Any, Any]] = dict()

def partisan_balance(Vf: float) -> float:
    balance: float = Vf - 0.5
    if Vf < 0.5:
        balance = -balance
    return balance

for sc, _data in conflicts_summary["with_proportionality"].items():
    report[sc] = dict()

    first_xx: str = list(_data.keys())[0]
    report[sc]["total"] = by_state[first_xx]["total"]

    conflicts_by_state = [(xx, partisan_balance(by_state[xx]['Vf']), _info['conflict-rate']) for xx, _info in _data.items()]
    conflicts_by_state.sort(key=lambda x: x[1])

    max_xx = max(_data.items(), key=lambda x: x[1]['delta'])[0]

    report[sc]["example"] = _data[max_xx]["example"]
    report[sc]["value"] = _data[max_xx]["value"]
    report[sc]["disproportionality"] = _data[max_xx]["disproportionality"]
    report[sc]["delta"] = _data[max_xx]["delta"]

    report[sc]['by-state'] = conflicts_by_state

report


{'efficiency_gap': {'total': 659997,
  'example': "('NY', 'upper', 'A0') / 007460000",
  'value': 0.005,
  'disproportionality': -0.1428,
  'delta': 0.14780000000000001,
  'by-state': [('NC', 0.005699999999999983, 0.010384895688919798),
   ('WI', 0.006800000000000028, 0.0035197129683922804),
   ('FL', 0.01629999999999998, 0.02937740626093755),
   ('MI', 0.01880000000000004, 0.02062130585442055),
   ('OH', 0.03620000000000001, 0.007393973002907589),
   ('IL', 0.0817, 0.7168108339886393),
   ('NY', 0.14780000000000004, 0.2433314090821625)]},
 'geometric_seats_bias': {'total': 659997,
  'example': "('NY', 'congress', 'D') / 031252500",
  'value': 0.0109,
  'disproportionality': -0.2466,
  'delta': 0.2575,
  'by-state': [('NC', 0.005699999999999983, 0.02901073792759664),
   ('WI', 0.006800000000000028, 0.005640934731521507),
   ('FL', 0.01629999999999998, 0.18232052569935925),
   ('MI', 0.01880000000000004, 0.02679103086832213),
   ('OH', 0.03620000000000001, 0.09843984139321846),
   ('IL'

Find conflict coverage

In [21]:
coverage = dict()
total_sum = sum(by_state[xx]["total"] for xx in states)

for sc in partisan_metrics[1:]:
    coverage[sc] = {
        "count": 0,
        "total": 0,
        "combos": 0
    }

    for combo, _data in conflicts["with_proportionality"][sc].items():
        xx, chamber, ensemble = combo
        coverage[sc]["total"] = total_sum
        # coverage[sc]["total"] = by_state[xx]["total"]
        coverage[sc]["count"] += _data["count"]
        if _data["count"] > 0:
            coverage[sc]["combos"] += 1

coverage

{'efficiency_gap': {'count': 680747, 'total': 4619979, 'combos': 165},
 'geometric_seats_bias': {'count': 1545812, 'total': 4619979, 'combos': 176},
 'seats_bias': {'count': 1510956, 'total': 4619979, 'combos': 176},
 'votes_bias': {'count': 1509410, 'total': 4619979, 'combos': 176},
 'mean_median_average_district': {'count': 1566306,
  'total': 4619979,
  'combos': 199},
 'lopsided_outcomes': {'count': 2042083, 'total': 4619979, 'combos': 189},
 'declination': {'count': 1309556, 'total': 4619979, 'combos': 165}}

Report the percentage of conflicts by metric and state.

In [22]:
print("Conflicts With Proportionality")
print("==============================")
print()

precision: int = 4
for sc, _data in report.items():
    name: str = _data["example"]
    sample: float = _data["value"]
    disp: float = _data["disproportionality"]

    by_metric_conflicts: int = coverage[sc]["count"]
    total_plans: int = coverage[sc]["total"]
    combos: int = coverage[sc]["combos"]

    by_state_output = [(state, f"{round(val1, precision):.2%}", f"{round(val2, precision):.2%}") for state, val1, val2 in _data['by-state']]


    print(f"{sc}:")
    print(
        f"  {by_metric_conflicts:,} of {total_plans:,} plans conflict ({by_metric_conflicts / total_plans:.1%} across {combos} of 231 = 7 x 3 x 11 state, chamber, and ensemble combinations."
    )
    if sc != 'declination':
        print(f"  Example: Map ({name}) has {sample:.2%} vs. disproportionality {disp:.2%}.")
    else:
        print(f"  Example: Map ({name}) has {sample:.4f} degrees vs. disproportionality {disp:.2%}.")
    print("  State, Deviation from 50-50 Balance, Conflict Rate")
    print(f"  {by_state_output}")
    print()

print()
print(
    f"Where a 'conflict' is when the sign of the metric is the *opposite* of the sign for simple 'disproportionality'."
)

Conflicts With Proportionality

efficiency_gap:
  680,747 of 4,619,979 plans conflict (14.7% across 165 of 231 = 7 x 3 x 11 state, chamber, and ensemble combinations.
  Example: Map (('NY', 'upper', 'A0') / 007460000) has 0.50% vs. disproportionality -14.28%.
  State, Deviation from 50-50 Balance, Conflict Rate
  [('NC', '0.57%', '1.04%'), ('WI', '0.68%', '0.35%'), ('FL', '1.63%', '2.94%'), ('MI', '1.88%', '2.06%'), ('OH', '3.62%', '0.74%'), ('IL', '8.17%', '71.68%'), ('NY', '14.78%', '24.33%')]

geometric_seats_bias:
  1,545,812 of 4,619,979 plans conflict (33.5% across 176 of 231 = 7 x 3 x 11 state, chamber, and ensemble combinations.
  Example: Map (('NY', 'congress', 'D') / 031252500) has 1.09% vs. disproportionality -24.66%.
  State, Deviation from 50-50 Balance, Conflict Rate
  [('NC', '0.57%', '2.90%'), ('WI', '0.68%', '0.56%'), ('FL', '1.63%', '18.23%'), ('MI', '1.88%', '2.68%'), ('OH', '3.62%', '9.84%'), ('IL', '8.17%', '100.00%'), ('NY', '14.78%', '99.99%')]

seats_bias:
  1,

Report conflicts by metric and state sorted by partisan balance

In [21]:
precision: int = 4
for sc, _data in report.items():
    x = [(state, round(val1, precision), round(val2, precision)) for state, val1, val2 in _data['by-state']]

    formatted_items = [f"({state}, {float(val1):>6.2%}, {float(val2):>7.2%})" for state, val1, val2 in x]
    print(f"{sc:>30}: {', '.join(formatted_items)}")



                efficiency_gap: (NC,  0.57%,   1.04%), (WI,  0.68%,   0.35%), (FL,  1.63%,   2.94%), (MI,  1.88%,   2.06%), (OH,  3.62%,   0.74%), (IL,  8.17%,  71.68%), (NY, 14.78%,  24.33%)
          geometric_seats_bias: (NC,  0.57%,   2.90%), (WI,  0.68%,   0.56%), (FL,  1.63%,  18.23%), (MI,  1.88%,   2.68%), (OH,  3.62%,   9.84%), (IL,  8.17%, 100.00%), (NY, 14.78%,  99.99%)
                    seats_bias: (NC,  0.57%,   2.86%), (WI,  0.68%,   0.56%), (FL,  1.63%,  14.48%), (MI,  1.88%,   2.67%), (OH,  3.62%,   8.53%), (IL,  8.17%,  99.98%), (NY, 14.78%,  99.86%)
                    votes_bias: (NC,  0.57%,   2.82%), (WI,  0.68%,   0.55%), (FL,  1.63%,  14.35%), (MI,  1.88%,   2.67%), (OH,  3.62%,   8.47%), (IL,  8.17%,  99.98%), (NY, 14.78%,  99.85%)
  mean_median_average_district: (NC,  0.57%,   9.00%), (WI,  0.68%,   0.55%), (FL,  1.63%,   5.97%), (MI,  1.88%,   2.45%), (OH,  3.62%,  19.44%), (IL,  8.17%,  99.92%), (NY, 14.78%, 100.00%)
             lopsided_outcomes: (NC,  0.

Report other conflicts

In [10]:
report2: Dict[str, Dict[Any, Any]] = dict()

for c in categories[1:]:
    report2[c] = dict()
    for sc, _data in conflicts_summary[c].items():
        report2[c][sc] = dict()

        first_xx: str = list(_data.keys())[0]
        report2[c][sc]["total"] = by_state[first_xx]["total"]

        conflicts_by_state = [(xx, partisan_balance(by_state[xx]['Vf']), _info['conflict-rate']) for xx, _info in _data.items()]
        conflicts_by_state.sort(key=lambda x: x[1])

        report2[c][sc]["example"] = _data[max_xx]["example"]
        report2[c][sc]['by-state'] = conflicts_by_state

In [11]:
titles: Dict[str, str] = {
    "with_proportionality": "N/A", 
    "within_symmetry": "Conflicts Within Symmetry",
    "within_packing_cracking": "Conflicts Within Packing & Cracking",
    "between_symmetry_and_packing_cracking": "Conflicts Between Symmetry & Packing & Cracking"
}

for c in categories[1:]:
    print(titles[c])
    print("=" * len(titles[c]))

    for sc, _data in report2[c].items():
        by_state_output = [(state, f"{round(val1, precision):.2%}", f"{round(val2, precision):.2%}") for state, val1, val2 in _data['by-state']]

        print(f"Subcategory: {sc}")
        print(f"Example: {_data['example']}")
        # print(f"Total: {_data['total']}")
        print("State, Deviation from 50-50 Balance, Conflict Rate")
        print(f"{by_state_output}")
        print()

    print()


Conflicts Within Symmetry
Subcategory: all
Example: ('NY', 'congress', 'A0') / 014170000
State, Deviation from 50-50 Balance, Conflict Rate
[('NC', '0.57%', '0.03%'), ('WI', '0.68%', '0.01%'), ('FL', '1.63%', '3.60%'), ('MI', '1.88%', '0.01%'), ('OH', '3.62%', '2.73%'), ('IL', '8.17%', '0.02%'), ('NY', '14.78%', '0.14%')]

Subcategory: seats_vs_votes
Example: None
State, Deviation from 50-50 Balance, Conflict Rate
[('NC', '0.57%', '0.00%'), ('WI', '0.68%', '0.00%'), ('FL', '1.63%', '0.00%'), ('MI', '1.88%', '0.00%'), ('OH', '3.62%', '0.00%'), ('IL', '8.17%', '0.00%'), ('NY', '14.78%', '0.00%')]


Conflicts Within Packing & Cracking
Subcategory: all
Example: ('NY', 'congress', 'C') / 036385000
State, Deviation from 50-50 Balance, Conflict Rate
[('NC', '0.57%', '11.79%'), ('WI', '0.68%', '0.47%'), ('FL', '1.63%', '18.74%'), ('MI', '1.88%', '0.32%'), ('OH', '3.62%', '70.08%'), ('IL', '8.17%', '4.98%'), ('NY', '14.78%', '0.02%')]


Conflicts Between Symmetry & Packing & Cracking
Subcategor